In [10]:
import cv2
import sys
import os
import numpy as np
from sys import platform
import argparse
import matplotlib.pyplot as plt
import pyopenpose as op
from IPython.display import clear_output as clear
path = 'D:/BaiduNetdiskDownload/openpose1/pic/'
vc = cv2.VideoCapture("D:/BaiduNetdiskDownload/openpose1/examples/media/3.20.mp4")  # 读入视频文件，命名cv
n = 1  # 计数
if vc.isOpened():  # 判断是否正常打开
    rval, frame = vc.read()
else:
    rval = False

timeF = 10  # 视频帧计数间隔频率
i = 0
data_pose = []
data_type = []
def clear_output():
    """
    clear output for both jupyter notebook and the console
    """
    os.system('cls' if os.name == 'nt' else 'clear')
    clear()

In [11]:
params = dict()
params["model_folder"] = "D:\\openpose\\models\\"
# Starting OpenPose
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

In [12]:
while rval:  # 循环读取视频帧
    rval, frame = vc.read()
    if (n % timeF == 0):  # 每隔timeF帧进行存储操作
        i += 1
        print('第', i, '张','     1:正常 2:转头 3:转身 4:传递 5:睡觉')

        # Process Image
        datum = op.Datum()
        imageToProcess = frame
        datum.cvInputData = imageToProcess
        opWrapper.emplaceAndPop(op.VectorDatum([datum]))
        plt.imshow(datum.cvOutputData)
        plt.show()
        print('当前人数（', datum.poseKeypoints.shape[0], '）:')
        type_pose = int(input())
        clear_output() # 清除输出
        if (type_pose == -2):
            break
        if(type_pose != -1):
            data_pose.append(datum.poseKeypoints[:, :, :2])
            for j in range(datum.poseKeypoints.shape[0]):
                data_type.append(type_pose)

        ##print(data)
        ##cv2.imwrite("D:/BaiduNetdiskDownload/openpose1/pic/image{}.jpg".format(i), frame)  # 存储为图像

    n = n + 1
    cv2.waitKey(1)

In [9]:
new_pose = np.concatenate(tuple(data_pose), axis=0)
new_type = np.array(data_type)
try:
    old_pose = np.load(path + 'data_pose.npy')
    old_type = np.load(path + 'data_type.npy')
    data_pose = np.append(old_pose, new_pose, axis=0)
    data_type = np.append(old_type, new_type, axis=0)
except:
    data_pose = new_pose
    data_type = new_type


np.save(path + 'data_pose', data_pose)
np.save(path + 'data_type', data_type)
print(data_type.shape, data_pose.shape)
vc.release()

ValueError: need at least one array to concatenate